In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key=<your_api_key>, 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("Who is Narendra modi.. ")
print(response.content)

Narendra Modi is an Indian politician who has been serving as the 14th Prime Minister of India since May 26, 2014. He is a member of the Bharatiya Janata Party (BJP) and is known for his strong nationalist and conservative views.

Early Life and Education:
Narendra Modi was born on September 17, 1950, in Vadnagar, a small town in the state of Gujarat, India. He was the third of six children born to Damodardas Mulchand Modi and Hiraben Modi. Modi's family was of the Ghanchi caste, a community of oil pressers. He completed his high school education in Vadnagar and later moved to Ahmedabad to pursue a degree in political science from the University of Delhi.

Career:
Modi joined the Rashtriya Swayamsevak Sangh (RSS), a Hindu nationalist organization, at a young age. He quickly rose through the ranks and became a full-time worker for the RSS in 1971. In 1987, Modi joined the BJP and became the party's general secretary in Gujarat. He played a key role in the BJP's victory in the 1995 Gujar

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-36827")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Lead Software Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc.

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm # Getting a prompt and passing it to the model (LLM)
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)
print(res.content)

[
  {
    "role": "Lead Software Engineer, ITC",
    "experience": "8+ years hands on experience in technical architecture, design, development & of demand or inventory planning products/applications",
    "skills": [
      "PL/SQL",
      "Cloud environments (AWS, Azure or GCP)",
      "Scrum and other Agile processes",
      "Jira, Git/SVN, Jenkins, CI/CD pipelines",
      "Integrated Business Planning solutions like O9",
      "Planning business domain, data domains, planning algorithms and solvers"
    ],
    "description": "Lead a team of engineers to develop solutions for Demand & Supply Planning capability. Advocate and implement business transformation solutions and best practices. Implement business solutions leveraging custom data platform and O9 platform."
  }
]


In [17]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Lead Software Engineer, ITC',
  'experience': '8+ years hands on experience in technical architecture, design, development & of demand or inventory planning products/applications',
  'skills': ['PL/SQL',
   'Cloud environments (AWS, Azure or GCP)',
   'Scrum and other Agile processes',
   'Jira, Git/SVN, Jenkins, CI/CD pipelines',
   'Integrated Business Planning solutions like O9',
   'Planning business domain, data domains, planning algorithms and solvers'],
  'description': 'Lead a team of engineers to develop solutions for Demand & Supply Planning capability. Advocate and implement business transformation solutions and best practices. Implement business solutions leveraging custom data platform and O9 platform.'}]

In [11]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [13]:
links = collection.query(query_texts=["Experience in Python", "Experience in React"],n_results=5)
links

{'ids': [['ce6f3530-d5f8-49ad-ab54-5772a8b1cfd3',
   'cfda480f-913a-4b84-94fe-77a7b4f016af',
   '53052381-a4e6-41aa-80c6-3c76f7a51ec6',
   '0fe9fec9-9de0-401d-9515-e8d78ba890e5',
   '6d6d0cce-59da-4bd2-a2fc-b9f4ca800f63'],
  ['b16a258f-dc21-4c19-a7eb-52faaefd5f0e',
   'da001209-c82c-4797-8aea-aef8704de604',
   'b0e6e36d-56f7-47df-a38b-03734075c174',
   '53648562-08db-4963-a7fc-574513679b5a',
   '7d5033af-9d8c-46f2-9345-12c83e94509a']],
 'distances': [[0.9968848007555503,
   1.0578701721493369,
   1.5701058255556164,
   1.6746084440071463,
   1.7099437952838776],
  [0.9630732819680135,
   1.0685748833873279,
   1.2124539626247859,
   1.4261935503356433,
   1.5300128428483581]],
 'metadatas': [[{'links': 'https://example.com/ml-python-portfolio'},
   {'links': 'https://example.com/python-portfolio'},
   {'links': 'https://example.com/magento-portfolio'},
   {'links': 'https://example.com/wordpress-portfolio'},
   {'links': 'https://example.com/devops-portfolio'}],
  [{'links': 'https://e

In [19]:
job = json_res[0]  
skills = job['skills']  
print(skills)

['PL/SQL', 'Cloud environments (AWS, Azure or GCP)', 'Scrum and other Agile processes', 'Jira, Git/SVN, Jenkins, CI/CD pipelines', 'Integrated Business Planning solutions like O9', 'Planning business domain, data domains, planning algorithms and solvers']


In [20]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}]]

In [21]:
job

{'role': 'Lead Software Engineer, ITC',
 'experience': '8+ years hands on experience in technical architecture, design, development & of demand or inventory planning products/applications',
 'skills': ['PL/SQL',
  'Cloud environments (AWS, Azure or GCP)',
  'Scrum and other Agile processes',
  'Jira, Git/SVN, Jenkins, CI/CD pipelines',
  'Integrated Business Planning solutions like O9',
  'Planning business domain, data domains, planning algorithms and solvers'],
 'description': 'Lead a team of engineers to develop solutions for Demand & Supply Planning capability. Advocate and implement business transformation solutions and best practices. Implement business solutions leveraging custom data platform and O9 platform.'}

In [22]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Nithish, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Nithish, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions for Demand & Supply Planning Capability

Dear Hiring Manager,

I came across the job description for a Lead Software Engineer, ITC, and I was impressed by the scope of the project. As a Business Development Executive at AtliQ, I'd like to introduce our company as a potential partner in developing solutions for Demand & Supply Planning capability.

At AtliQ, we specialize in providing AI and software consulting services that facilitate seamless integration of business processes through automated tools. Our team of experts has a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

Our expertise in cloud environments (AWS, Azure, or GCP), Scrum, and Agile processes, as well as our experience with Jira, Git/SVN, Jenkins, and CI/CD pipelines, aligns perfectly with your requirements. We've also worked on Integrated Business Planning solutions and have a